# Quantum Langevin Equations for Modulated Optoelectromechanical Systems

## Initialization

In [1]:
# dependencies
from IPython.display import display, Math
from sympy import exp, Function, I, im, latex, re, sqrt, symbols
from sympy.physics.quantum import Commutator, Dagger, hbar, Operator

In [54]:
# laser
A_l, kappa, omega_l = symbols('A_{l}, \\kappa, \\omega_{l}', real=True, positive=True)
# cavity
gamma_0, l, omega_c = symbols('\\gamma_{0}, l, \\omega_{c}', real=True, positive=True)
Delta_0 = symbols('\\Delta^{(0)}', real=True)
# mechanical
gamma_1, m, omega_0 = symbols('\\gamma_{1}, m, \\omega_{0}', real=True, positive=True)
# LC circuit
C_b, C_p, d, L, omega_1, r = symbols('C_{b}, C_{p}, d, L, \\omega_{1}, r', real=True, positive=True)
A_v, V_DC = symbols('A_{v}, V_{DC}', real=True)
# interaction
G, g_0, g_1, g_1_0, g_1_1 = symbols('G, g_{0}, g_{1}, g_{1}^{(0)}, g_{1}^{(1)}', real=True, positive=True)
# time 
t = symbols('t', real=True, positive=True)

# modes
a_t = Function('\\hat{a}', commutative=False)(t)
b_0_t = Function('\\hat{b}_{0}', commutative=False)(t)
b_1_t = Function('\\hat{b}_{1}', commutative=False)(t)

# quadratures
q_0_t = Function('\\hat{q}_{0}', commutative=False)(t)
p_0_t = Function('\\hat{p}_{0}', commutative=False)(t)
q_1_t = Function('\\hat{q}_{1}', commutative=False)(t)
p_1_t = Function('\\hat{p}_{1}', commutative=False)(t)

# dimensionless quadratures
Q_0_t = Function('\\hat{Q}_{0}', commutative=False)(t)
P_0_t = Function('\\hat{P}_{0}', commutative=False)(t)
Q_1_t = Function('\\hat{Q}_{1}', commutative=False)(t)
P_1_t = Function('\\hat{P}_{1}', commutative=False)(t)

# fluctuations
a_in_t = Function('\\hat{a}^{(in)}', commutative=False)(t)
b_0_in_t = Function('\\hat{b}_{0}^{(in)}', commutative=False)(t)
b_1_in_t = Function('\\hat{b}_{1}^{(in)}', commutative=False)(t)

## Capacitance

In [3]:
# capacitance
expr_C_num = C_p * (1 - q_0_t / d) + C_b
expr_C_den = 1 - q_0_t / d
display(Math('\\frac{1}{C} = ' + latex(expr_C_den / expr_C_num)))

# substitute participation ratio
expr_C_num = (expr_C_num * r / C_b).subs(C_p, C_b * (1 - r) / r).expand() * C_b / r
expr_C_inv = expr_C_den / expr_C_num
display(Math('\\Rightarrow \\frac{1}{C} = ' + latex(expr_C_inv)))

# binomial approximation
expr_C_inv_approx = expr_C_inv.subs((1 + r * q_0_t / d - q_0_t / d)**-1, 1 - r * q_0_t / d + q_0_t / d).expand()
display(Math('\\Rightarrow \\frac{1}{C} \\approx ' + latex(expr_C_inv_approx)))

# neglect quadratic terms
expr_C_inv_approx = expr_C_inv_approx.subs(q_0_t**2 / d**2, 0).expand()
display(Math('\\Rightarrow \\frac{1}{C} \\approx ' + latex(expr_C_inv_approx)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Inductance

In [4]:
# inductance
expr_L_inv = omega_1**2 * (expr_C_num / expr_C_den)
display(Math('\\frac{1}{L} = ' + latex(expr_L_inv)))

# binomial approximation
expr_L_inv_approx = expr_L_inv.subs((1 - q_0_t / d)**-1, 1 + q_0_t / d).expand()
display(Math('\\Rightarrow \\frac{1}{L} \\approx ' + latex(expr_L_inv_approx)))

# neglect quadratic terms
expr_L_inv_approx = expr_L_inv_approx.subs((q_0_t / d)**2, 0)
display(Math('\\Rightarrow \\frac{1}{L} \\approx ' + latex(expr_L_inv_approx)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Hamiltonian in the Interaction Picture

In [5]:
# laser Hamiltonian
expr_H_l = I * hbar * A_l * (Dagger(a_t) * exp(- I * omega_l * t) - a_t * exp(I * omega_l * t))
# cavity Hamiltonian
expr_H_c = hbar * omega_c * Dagger(a_t) * a_t
# mechanical Hamiltonian
expr_H_0 = m * omega_0**2 * q_0_t**2 / 2 + p_0_t**2 / 2 / m
# LC circuit Hamiltonian
expr_H_1 = r * q_1_t**2 / 2 / C_b + p_1_t**2 / 2 / L - q_1_t * V_DC
# interaction Hamiltonians
expr_H_om = - hbar * omega_c / l * q_0_t * Dagger(a_t) * a_t
expr_H_em = - r**2 / 2 / C_b / d * q_0_t * q_1_t**2
# total Hamiltonian
expr_H_tot = expr_H_c + expr_H_0 + expr_H_om + expr_H_1 + expr_H_em + expr_H_l
display(Math('H_{tot} = ' + latex(expr_H_tot)))

# hamiltonian in the interaction picture
list_subs = [
    (a_t, a_t * exp(- I * omega_l * t)), 
    (omega_c / l, G), 
    (omega_c, omega_l + Delta_0)
]
expr_H_int = expr_H_tot.subs(list_subs).expand() - hbar * omega_l * Dagger(a_t) * a_t
display(Math('\\Rightarrow H_{int} = ' + latex(expr_H_int)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Hamiltonian in terms of Dimensionless Quadratures

In [6]:
# zero point fluctuations
expr_q_0_zpf = sqrt(hbar / m / omega_0)
expr_p_0_zpf = sqrt(hbar * m * omega_0)
expr_q_1_zpf = sqrt(hbar / L / omega_1)
expr_p_1_zpf = sqrt(hbar * L * omega_1)
list_subs = [
    (q_0_t, expr_q_0_zpf * Q_0_t), 
    (p_0_t, expr_p_0_zpf * P_0_t), 
    (q_1_t, expr_q_1_zpf * Q_1_t), 
    (p_1_t, expr_p_1_zpf * P_1_t)
]
expr_H_dim = expr_H_int.subs(list_subs).expand()
display(Math('q_{0_{zpf}} = ' + latex(expr_q_0_zpf) + ',\\quad p_{0_{zpf}} = ' + latex(expr_p_0_zpf)))
display(Math('q_{1_{zpf}} = ' + latex(expr_q_1_zpf) + ',\\quad p_{1_{zpf}} = ' + latex(expr_p_1_zpf)))
# display(Math('\\Rightarrow H_{dim} = ' + latex(expr_H_dim)))

# coupling coefficients
expr_g_0 = G * expr_q_0_zpf / sqrt(2)
expr_g_1_0 = expr_q_0_zpf * expr_q_1_zpf**2 * r**2 / 4 / sqrt(2) / hbar / C_b / d
expr_A_v = expr_q_1_zpf * V_DC / sqrt(2) / hbar
expr_L_inv_dim = expr_L_inv.subs(q_0_t, expr_q_0_zpf * Q_0_t)
expr_L_inv_approx_dim = expr_L_inv_approx.subs(q_0_t, expr_q_0_zpf * Q_0_t)
list_subs = [
    (sqrt(2) * expr_g_0, sqrt(2) * g_0), 
    (2 * sqrt(2) * expr_g_1_0, 2 * sqrt(2) * g_1_0), 
    (sqrt(2) * hbar * expr_A_v, sqrt(2) * hbar * A_v), 
    (1 / L, expr_L_inv_approx_dim)
]
expr_H_dim = expr_H_dim.subs(list_subs).expand()
display(Math('g_{0} = ' + latex(expr_g_0) + ',\\quad g_{1}^{(0)} = ' + latex(expr_g_1_0) + ',\\quad A_{v} = ' + latex(expr_A_v)))
display(Math('\\frac{1}{L} = ' + latex(expr_L_inv_dim) + ' \\approx ' + latex(expr_L_inv_approx_dim)))
# display(Math('\\Rightarrow H_{dim} \\approx ' + latex(expr_H_dim)))

# effective coupling constant
expr_g_1_1 = r * omega_1 / d * expr_q_0_zpf / 4 / sqrt(2)
list_subs = [
    (2 * sqrt(2) * expr_g_1_1, 2 * sqrt(2) * g_1_1), 
    (g_1_0, g_1 + g_1_1)
]
expr_H_dim = expr_H_dim.subs(list_subs).expand().collect([I * hbar * A_l, hbar * omega_0, hbar * omega_1])
display(Math('g_{1}^{(1)} = ' + latex(expr_g_1_1) + ',\\quad g_{1} = g_{1}^{(0)} - g_{1}^{(1)}'))
display(Math('\\Rightarrow H_{dim} \\approx ' + latex(expr_H_dim)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Hamiltonian in terms of the Modes

In [7]:
# energies and quadratures
list_subs = [
    (Q_0_t**2 / 2 + P_0_t**2 / 2, Dagger(b_0_t) * b_0_t),
    (Q_1_t**2 / 2 + P_1_t**2 / 2, Dagger(b_1_t) * b_1_t),
    (Q_0_t, (Dagger(b_0_t) + b_0_t) / sqrt(2)), 
    (P_0_t, I * (Dagger(b_0_t) - b_0_t) / sqrt(2)), 
    (Q_1_t, (Dagger(b_1_t) + b_1_t) / sqrt(2)), 
    (P_1_t, I * (Dagger(b_1_t) - b_1_t) / sqrt(2))
]
expr_H = expr_H_dim.subs(list_subs).collect([I * hbar * A_l, - hbar * A_v])
display(Math('H = ' + latex(expr_H)))

<IPython.core.display.Math object>

## Heisenberg Equations of Motion

In [53]:
# mode rates
expr_da_dt_t = (- I / hbar * Commutator(a_t, expr_H)).doit().expand()
expr_db_0_dt_t = (- I / hbar * Commutator(b_0_t, expr_H)).doit().expand()
expr_db_1_dt_t = (- I / hbar * Commutator(b_1_t, expr_H)).doit().expand()

# commutation relations
list_subs = [
    (a_t * Dagger(a_t), 1 + Dagger(a_t) * a_t), 
    (a_t * b_0_t, b_0_t * a_t), 
    (a_t * Dagger(b_0_t), Dagger(b_0_t) * a_t), 
    (a_t * b_1_t, b_1_t * a_t), 
    (a_t * Dagger(b_1_t), Dagger(b_1_t) * a_t),
    (Dagger(a_t) * b_0_t, b_0_t * Dagger(a_t)), 
    (Dagger(a_t) * Dagger(b_0_t), Dagger(b_0_t) * Dagger(a_t)), 
    (Dagger(a_t) * b_1_t, b_1_t * Dagger(a_t)), 
    (Dagger(a_t) * Dagger(b_1_t), Dagger(b_1_t) * Dagger(a_t)),
    (b_0_t * Dagger(b_0_t), 1 + Dagger(b_0_t) * b_0_t),
    (b_0_t * b_1_t, b_1_t * b_0_t), 
    (b_0_t * Dagger(b_1_t), Dagger(b_1_t) * b_0_t),
    (Dagger(b_0_t) * b_1_t, b_1_t * Dagger(b_0_t)), 
    (Dagger(b_0_t) * Dagger(b_1_t), Dagger(b_1_t) * Dagger(b_0_t)),
    (b_1_t * Dagger(b_1_t), 1 + Dagger(b_1_t) * b_1_t),
]
# cavity mode
expr_da_dt_t = expr_da_dt_t.subs(list_subs).expand()
expr_da_dt_t = expr_da_dt_t.subs(list_subs).expand().collect(I * g_0)
display(Math('\\frac{d\\dot{a}}{dt}(t) = ' + latex(expr_da_dt_t)))
# mechanical mode
expr_db_0_dt_t = expr_db_0_dt_t.subs(list_subs).expand()
expr_db_0_dt_t = expr_db_0_dt_t.subs(list_subs).expand()
expr_db_0_dt_t = expr_db_0_dt_t.subs(list_subs).expand().collect(I * g_1)
display(Math('\\frac{d\\dot{b}_0}{dt}(t) = ' + latex(expr_db_0_dt_t)))
# LC circuit mode
expr_db_1_dt_t = expr_db_1_dt_t.subs(list_subs).expand()
expr_db_1_dt_t = expr_db_1_dt_t.subs(list_subs).expand()
expr_db_1_dt_t = expr_db_1_dt_t.subs(list_subs).expand().collect(2 * I * g_1)
display(Math('\\frac{d\\dot{b}_1}{dt}(t) = ' + latex(expr_db_1_dt_t)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Quantum Langevin Equations

In [55]:
# cavity decay and vaccum fluctuations
expr_da_dt_t_io = expr_da_dt_t - kappa * a_t + sqrt(2 * kappa) * a_in_t
display(Math('\\frac{d\\dot{a}}{dt}(t) = ' + latex(expr_da_dt_t_io)))
# mechanical damping and thermal fluctuations
expr_db_0_dt_t_io = expr_db_0_dt_t - gamma_0 * b_0_t + sqrt(2 * gamma_0) * b_0_in_t
display(Math('\\frac{d\\dot{b}_0}{dt}(t) = ' + latex(expr_db_0_dt_t_io)))
# electrical damping and thermal fluctuations
expr_db_1_dt_t_io = expr_db_1_dt_t - gamma_1 * b_1_t + sqrt(2 * gamma_1) * b_1_in_t
display(Math('\\frac{d\\dot{b}_1}{dt}(t) = ' + latex(expr_db_1_dt_t_io)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>